In [ ]:
%pip install plotly "ipywidgets>=7.0.0"

In [3]:
import plotly.graph_objects as go

import numpy as np
np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])

scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#bae2be'
        s[i] = 20
        with f.batch_update():
            scatter.marker.color = c
            scatter.marker.size = s


scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                         

In [12]:
import os
import pandas as pd

if os.getcwd() == "/drive":
    data = pd.read_parquet('./dim_medium_authors.parquet')
else:
    data = pd.read_parquet('../../output/marts/dim_medium_authors.parquet')

data

,author_name,author_url,extracted_at,extracted_at_epoch,extraction_id,num_followers,short_bio,linkedin_username,spacy_location,twitter_handle,is_author_based_in_netherlands
0,Stephen David-Williams,https://blog.det.life/@sdw-online,1690524796,1690524796,28ee661d-41e6-4337-b467-be30344b76d7.json,443,I am a developer with 5+ years of data enginee...,,,,False
1,Jim Todd,https://blog.devgenius.io/@jimtodd92,1684149607,1684149607,a490a299-9881-4388-9586-2a5f19630b4d.json,10,Data Engineer who loves data and to learn more...,,,,False
2,Ravi Kumar,https://blog.devgenius.io/@ravikumar10593,1689920022,1689920022,cfb017cb-184e-4f7c-9669-21b5f02b43f1.json,131,"Hey Guys!, I am here to share my learning abou...",,,,False
3,Pavel Makarichev,https://blog.opendatadiscovery.org/@vixtir90,1684488989,1684488989,ef483d50-8156-4964-ba52-72cf6c9eadb8.json,18,,,,,False
4,Kalpan Shah,https://faun.pub/@developershome,1684149607,1684149607,a490a299-9881-4388-9586-2a5f19630b4d.json,144,Senior Data Engineer | Developer | Data Enthus...,,,,False
...,...,...,...,...,...,...,...,...,...,...,...
1805,Federico Lavatori,https://medium.com/@federico.lavatori,1684149607,1684149607,a490a299-9881-4388-9586-2a5f19630b4d.json,12,Data Scientist | Data Engineer @HYPE,,,,False
1806,Adevinta,https://medium.com/@adevinta,1687511557,1687511557,125749c7-5db7-43e8-9d01-0cea39d5a01f.json,423,Creating perfect matches on the world’s most t...,,,,False
1807,Marcos Alcozer,https://medium.com/@marcos_alcozer,1684149607,1684149607,a490a299-9881-4388-9586-2a5f19630b4d.json,44,K12 analytics engineer,,,,False
1808,YouJin,https://medium.com/@youjin-lim,1684149607,1684149607,a490a299-9881-4388-9586-2a5f19630b4d.json,0,,,,,False
